In [14]:
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import ast

In [ ]:
models = {
    'nb': GaussianNB(),
    'svm': SVC(),
    'rf': RandomForestClassifier()
}

In [2]:
from collections import Counter

def get_rare_tokens(df: pd.DataFrame, threshold: int=2) -> set:
    all_sentences = df['tokens'].tolist()
    all_tokens = [token for sentence in all_sentences for token in sentence]
    token_counts = Counter(all_tokens)
    rare_tokens = {token for token, count in token_counts.items() if count < threshold}
    return rare_tokens

# Predict on the original splits

In [3]:
df = pd.read_csv("data/preprocessed_data.csv")
df['tokens'] = df['tokens'].apply(ast.literal_eval)

df_train = df[df['dataset'] == 'train']
df_train.head()

,sentence,topic,sentiment,dataset,tokens
0,slide giáo trình đầy đủ .,1,2,train,"[slide, giáo_trình, đầy_đủ]"
1,"nhiệt tình giảng dạy , gần gũi với sinh viên .",0,2,train,"[nhiệt_tình, giảng_dạy, gần_gũi, sinh_viên]"
2,đi học đầy đủ full điểm chuyên cần .,1,0,train,"[đi, học, đầy_đủ, full, chuyên_cần]"
3,chưa áp dụng công nghệ thông tin và các thiết ...,0,0,train,"[áp_dụng, công_nghệ, thông_tin, thiết_bị, giản..."
4,"thầy giảng bài hay , có nhiều bài tập ví dụ ng...",0,2,train,"[thầy, giảng, bài_tập, ví_dụ, lớp]"


In [4]:
rare_tokens = get_rare_tokens(df_train, 2)

print(f"The number of rare tokens is: {len(rare_tokens)}")
print(f"Some rare tokens are: {list(rare_tokens)[:5]}")

The number of rare tokens is: 1623
Some rare tokens are: ['đế', 'gắn_bó', 'bơi', 'ngày_nay', 'thời_gian_biểu']


In [5]:
df['popular'] = df['tokens'].apply(lambda x: [token for token in x if token not in rare_tokens])
df.head()

,sentence,topic,sentiment,dataset,tokens,popular
0,slide giáo trình đầy đủ .,1,2,train,"[slide, giáo_trình, đầy_đủ]","[slide, giáo_trình, đầy_đủ]"
1,"nhiệt tình giảng dạy , gần gũi với sinh viên .",0,2,train,"[nhiệt_tình, giảng_dạy, gần_gũi, sinh_viên]","[nhiệt_tình, giảng_dạy, gần_gũi, sinh_viên]"
2,đi học đầy đủ full điểm chuyên cần .,1,0,train,"[đi, học, đầy_đủ, full, chuyên_cần]","[đi, học, đầy_đủ, chuyên_cần]"
3,chưa áp dụng công nghệ thông tin và các thiết ...,0,0,train,"[áp_dụng, công_nghệ, thông_tin, thiết_bị, giản...","[áp_dụng, công_nghệ, thông_tin, thiết_bị, giản..."
4,"thầy giảng bài hay , có nhiều bài tập ví dụ ng...",0,2,train,"[thầy, giảng, bài_tập, ví_dụ, lớp]","[thầy, giảng, bài_tập, ví_dụ, lớp]"


In [6]:
select_cols = ['popular', 'topic', 'sentiment']

df_train = df[df['dataset'] == 'train'][select_cols]
df_valid = df[df['dataset'] == 'valid'][select_cols]
df_test = df[df['dataset'] == 'test'][select_cols]

In [8]:
train_sentences = [" ".join(sent) for sent in df_train['popular']]
valid_sentences = [" ".join(sent) for sent in df_valid['popular']]
test_sentences = [" ".join(sent) for sent in df_test['popular']]

In [9]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(train_sentences)
X_valid = vectorizer.transform(valid_sentences)
X_test = vectorizer.transform(test_sentences)

## Predict topics

In [16]:
y_train = df_train['topic']
y_valid = df_valid['topic']
y_test = df_test['topic']